### IMPORTS

In [1]:
import pandas as pd
import numpy as np
from sqlalchemy import create_engine, inspect
import datetime
from dateutil.relativedelta import relativedelta
from Roster import *
from gspread_pandas import Spread, conf

### CREDENCIALES

In [2]:
cred = conf.get_config('C:\\Users\\santiago.curat\\Pandas\\PEYA', 'PedidosYa-8b8c4d19f61c.json')

### CONSTANTES

In [3]:
# Fechas
today = datetime.date.today()
ilm = str((today - relativedelta(months=1)).replace(day=1))
flm = str(today - relativedelta(days=today.day))

### GOOGLE SHEETS

In [6]:
# Partners PEYA
sheet_id = '1HmAvHYbJJa3JyRTgRJGKXataQB_TvFwNzyo8a6qXd2o'
wks_name = 'Partners PEYA'
sheet = Spread(sheet_id, wks_name, config=cred)
partners = sheet.sheet_to_df(index=0,header_rows=1)

In [7]:
# Reporte KAM
sheet_id = '1JNywQTVzEQKRwqrJRkpzjiXx5Ly-FldtBMfeSYHuL7w'
wks_name = 'KAM'
sheet = Spread(sheet_id, wks_name, config=cred)
reporte_kams = sheet.sheet_to_df(index=0,header_rows=1)

In [8]:
# Franchises KAM
franchises_kam = [x for x in reporte_kams[reporte_kams.columns[0]].to_list() if x is not '']
# KAMs
kams = [x.upper() for x in reporte_kams[reporte_kams.columns[1]].to_list() if x is not '']

### QUERIES

In [34]:
# 1.1GB
q_orders = '''SELECT c.city_name AS City,
       a.area_name AS Area,
       o.business_type.business_type_name AS Business,
       COUNT(DISTINCT o.order_id) AS Total,
       COUNT(DISTINCT CASE WHEN o.order_status = 'CONFIRMED' THEN o.order_id ELSE NULL END) AS Confirmed,
       COUNT(DISTINCT CASE WHEN o.order_status = 'CONFIRMED' AND o.with_logistics THEN o.order_id ELSE NULL END) AS Logistic,
       COUNT(DISTINCT CASE WHEN o.order_status IN ('PENDING','REJECTED','CANCELLED') THEN o.order_id ELSE NULL END) AS FR_Num,
       COUNT(DISTINCT CASE WHEN o.order_status IN ('PENDING','REJECTED','CANCELLED') AND o.fail_rate_owner = 'Restaurant' THEN o.order_id ELSE NULL END) AS VFR_Num,
       SUM(CASE WHEN o.order_status = 'CONFIRMED' THEN o.commission_amount ELSE 0 END) AS Revenue,
       SUM(CASE WHEN o.with_logistics AND o.order_status = 'CONFIRMED' THEN o.commission_amount ELSE 0 END) AS Log_Revenue,
       SUM(CASE WHEN o.with_logistics AND o.order_status = 'CONFIRMED' THEN o.shipping_amount_no_discount ELSE 0 END) AS DF_Log
FROM `peya-bi-tools-pro.il_core.fact_orders` AS o
LEFT JOIN `peya-bi-tools-pro.il_core.dim_area` AS a ON o.address.area.id = a.area_id
LEFT JOIN `peya-bi-tools-pro.il_core.dim_city` AS c ON o.city.city_id = c.city_id
WHERE o.country_id = 3
      AND o.registered_date BETWEEN DATE('{0}') AND DATE('{1}')
GROUP BY 1,2,3'''.format(ilm,flm)

# 1.1GB
q_vl = '''SELECT c.city_name AS City,
       a.area_name AS Area,
       o.business_type.business_type_name AS Business,
       IFNULL(COUNT(DISTINCT CASE WHEN lo.is_vendor_late_10 = 1 THEN o.order_id ELSE NULL END),0) AS VL_Numerador,
       IFNULL(COUNT(DISTINCT CASE WHEN lo.is_vendor_late_nn = 1 THEN o.order_id ELSE NULL END),0) AS VL_Denominador,
       SUM(lo.timings.actual_delivery_time*1.00/60) AS DT_Numerador,
       COUNT(DISTINCT lo.peya_order_id) AS DT_Denominador
FROM `peya-bi-tools-pro.il_logistics.fact_logistic_orders` AS lo
LEFT JOIN `peya-bi-tools-pro.il_core.fact_orders` AS o ON lo.peya_order_id = o.order_id
LEFT JOIN `peya-bi-tools-pro.il_core.dim_area` AS a ON o.address.area.id = a.area_id
LEFT JOIN `peya-bi-tools-pro.il_core.dim_city` AS c ON o.city.city_id = c.city_id
WHERE lo.created_date_local BETWEEN DATE('{0}') AND DATE('{1}')
      AND o.registered_date BETWEEN DATE('{0}') AND DATE('{1}')
      AND o.country_id = 3
GROUP BY 1,2,3'''.format(ilm,flm)

# 8.1MB
q_online = '''SELECT p.city.name AS City,
       a.area_name AS Area,
       p.business_type.business_type_name AS Business,
       COUNT(DISTINCT CASE WHEN pm.is_online THEN p.partner_id ELSE NULL END) AS Online,
       COUNT(DISTINCT CASE WHEN pm.is_churned THEN p.partner_id ELSE NULL END) AS Churns,
       COUNT(DISTINCT CASE WHEN pm.is_online AND pm.has_confirmed_orders = FALSE THEN p.partner_id ELSE NULL END) AS Zombies
FROM `peya-bi-tools-pro.il_core.fact_partners_monthly` AS pm
LEFT JOIN `peya-bi-tools-pro.il_core.dim_partner` AS p ON pm.restaurant_id = p.partner_id
LEFT JOIN `peya-bi-tools-pro.il_core.dim_area` AS a ON p.address.area_id = a.area_id
WHERE p.country_id = 3
      AND DATE(pm.full_date) BETWEEN DATE('{0}') AND DATE('{1}')
GROUP BY 1,2,3'''.format(ilm,flm)

# 50.3MB
q_cw = '''SELECT p.salesforce_id AS Grid,
       sfo.Business_Type AS Tipo,
       sfc.Name__c AS Nombre,
       cw.account_owner AS Owner,
       MAX(sfc.Commission) AS Comision
FROM `peya-bi-tools-pro.il_core.fact_sales_executives_closed_won_v2` AS cw
LEFT JOIN `peya-bi-tools-pro.il_core.dim_partner` AS p ON cw.grid = p.salesforce_id
LEFT JOIN `peya-data-origins-pro.cl_salesforce.opportunity` AS sfo ON cw.oportunity_unique_id = sfo.Id
LEFT JOIN `peya-data-origins-pro.cl_salesforce.contract` AS sfc ON sfo.Id = sfc.Id_Opportunity
WHERE DATE(cw.closed_date) BETWEEN DATE('{0}') AND DATE('{1}')
      AND p.country_id = 3
      AND sfo.Business_Type IN ('New Business','Win Back','Franchise Extension','New Bussiness')
GROUP BY 1,2,3,4'''.format(ilm,flm)

# 862MB
q_nps = '''SELECT c.city_name AS City,
       a.area_name AS Area,
       o.business_type.business_type_name AS Business,
       1.0 * ((COUNT(DISTINCT CASE WHEN (q.nps_group = 'Promoter') THEN CASE WHEN q.response_id IS NULL THEN '0' ELSE q.response_id END ELSE NULL END)) - (COUNT(DISTINCT CASE WHEN (q.nps_group = 'Detractor') THEN CASE WHEN q.response_id IS NULL THEN '0' ELSE q.response_id END ELSE NULL END))) AS NPS_AO_Numerador,
       1.0 * (CASE WHEN (COUNT(DISTINCT CASE WHEN q.response_id IS NULL THEN '0' ELSE q.response_id END)) = 0 THEN 1 ELSE (COUNT(DISTINCT CASE WHEN q.response_id IS NULL THEN '0' ELSE q.response_id END)) END) AS NPS_AO_Denominador
FROM `peya-bi-tools-pro.il_nps.fact_qualtrics` AS q
LEFT JOIN `peya-bi-tools-pro.il_core.fact_orders` AS o ON q.order_id = o.order_id
LEFT JOIN `peya-bi-tools-pro.il_core.dim_area` AS a ON o.address.area.id = a.area_id
LEFT JOIN `peya-bi-tools-pro.il_core.dim_city` AS c ON o.city.city_id = c.city_id
WHERE o.country_id = 3
      AND o.registered_date BETWEEN DATE('{0}') AND DATE('{1}')
GROUP BY 1,2,3'''.format(ilm,flm)

In [35]:
# Descargo la data
hue_nps = pd.io.gbq.read_gbq(q_nps, project_id='peya-argentina', dialect='standard')

Downloading: 100%|█████████████████████████████████████████████████████████████| 3230/3230 [00:01<00:00, 2191.70rows/s]


In [36]:
hue_orders = pd.io.gbq.read_gbq(q_orders, project_id='peya-argentina', dialect='standard')

Downloading: 100%|███████████████████████████████████████████████████████████| 10532/10532 [00:02<00:00, 3555.01rows/s]


In [37]:
hue_vl = pd.io.gbq.read_gbq(q_vl, project_id='peya-argentina', dialect='standard')

Downloading: 100%|█████████████████████████████████████████████████████████████| 9264/9264 [00:02<00:00, 4321.08rows/s]


In [38]:
hue_online = pd.io.gbq.read_gbq(q_online, project_id='peya-argentina', dialect='standard')

Downloading: 100%|█████████████████████████████████████████████████████████████| 3773/3773 [00:01<00:00, 2320.14rows/s]


In [39]:
hue_cw = pd.io.gbq.read_gbq(q_cw, project_id='peya-argentina', dialect='standard')

Downloading: 100%|█████████████████████████████████████████████████████████████| 2807/2807 [00:01<00:00, 2031.45rows/s]


In [40]:
# Copio las bases
nps = hue_nps.copy()
orders = hue_orders.copy()
vl = hue_vl.copy()
online = hue_online.copy()
cw = hue_cw.copy()

### TRABAJO

#### TRABAJO LOS TARGETS

In [41]:
# Trabajo NPS
val = [i for i in nps.columns if i not in ['City','Area','Business']]
nps[val] = nps[val].astype(float)
# Trabajo Orders
val = [i for i in orders.columns if i not in ['City','Area','Business']]
orders[val] = orders[val].astype(float)
# Trabajo VL
val = [i for i in vl.columns if i not in ['City','Area','Business']]
vl[val] = vl[val].astype(float)
# Trabajo Online
val = [i for i in online.columns if i not in ['City','Area','Business']]
online[val] = online[val].astype(float)

In [43]:
# Uno las tablas
targets = orders.merge(nps,on=['City','Area','Business'],how='outer')
targets = targets.merge(vl,on=['City','Area','Business'],how='outer')
targets = targets.merge(online,on=['City','Area','Business'],how='outer')
targets.replace([np.nan,np.inf,-np.inf],0,inplace=True)

In [44]:
# Coloco el Feudo y Reino
targets['Feudo'] = targets.apply(feudos,axis=1)
targets['Reino'] = targets.apply(reinos,axis=1)

In [45]:
# Divido según Vertical
targets['Restaurants/NV'] = targets['Business'].apply(lambda x: x if x in ['Restaurant','Coffee'] else 'NV')

In [46]:
# Hago la PT Final
val = [i for i in targets.columns if i not in ['City','Area','Business','Feudo','Reino','Restaurants/NV']]
targets = targets.pivot_table(index=['Reino','Feudo','Restaurants/NV'],values=val,fill_value=0,aggfunc='sum').reset_index()

#### TRABAJO LOS CW

In [20]:
# Agrego extras a KAM
kams = kams + ['LUCRECIA BORIA']

In [21]:
# Filtro columnas a usar de Partners
cols = ['Grid','Name','Franchise','City','Area','Feudo','Reino','KAM','Concept','Logistic','Business','Main_Cuisine']
partners = partners[cols].copy()

In [22]:
# Doy formato a las columnas
cw['Grid'] = cw['Grid'].astype(str)
partners['Grid'] = partners['Grid'].astype(str)
# Uno las tablas
cw = cw.merge(partners,on=['Grid'],how='left')
# Coloco los Owner KAM
cw['Owner'] = cw['Owner'].str.upper()
cw['Owner KAM'] = cw['Owner'].apply(lambda x: 'Si' if x in kams else 'No')

### CARGA

In [30]:
# Carga Contratos
sheet_id = '1sr3-eriuc2hZEWWTK_1jt1XDgNnx91vWIsM99mMV4BE'
wks_name = 'CW BQ'
sheet = Spread(sheet_id, wks_name, config=cred)
sheet.df_to_sheet(cw, index=False, sheet=wks_name, replace=True)

In [35]:
# Carga Contratos
sheet_id = '1FN0iqV38uYj8tOT1ctCzWlCMO4ABaUIiuYiU6BuwdaE'
wks_name = 'CW BQ'
sheet = Spread(sheet_id, wks_name, config=cred)
sheet.df_to_sheet(cw, index=False, sheet=wks_name, replace=True)

In [50]:
# Carga Regiones
sheet_id = '1fgwyXK8AmIn02rjpKQHLQQEK97uh2iRsTCUAF29A9cU'
wks_name = 'Crudo BQ'
sheet = Spread(sheet_id, wks_name, config=cred)
sheet.df_to_sheet(targets, index=False, sheet=wks_name, replace=True)

In [51]:
# Carga AMBA
sheet_id = '1C9DNHC8BNf_SCH84SEfVQJ1nbFHxCdpDpMffGSVil2I'
wks_name = 'Crudo BQ'
sheet = Spread(sheet_id, wks_name, config=cred)
sheet.df_to_sheet(targets, index=False, sheet=wks_name, replace=True)